## Demo: Tokenisierung und kontextabhängige Embeddings mit BERT

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
from torch.nn.functional import cosine_similarity

In [2]:
s1 = "Ongchoi is delicious sautéed with garlic."
s2 = "The farmer harvested the ongchoi before the rainy season."

In [3]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [4]:
s1_tokenized = tokenizer(s1, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(s1_tokenized['input_ids'][0])
print(tokens)


['[CLS]', 'on', '##gc', '##ho', '##i', 'is', 'delicious', 'sa', '##ute', '##ed', 'with', 'garlic', '.', '[SEP]']


| Token   | Bedeutung                                                                                                                                   |
| ------- | ------------------------------------------------------------------------------------------------------------------------------------------- |
| `[CLS]` | *Classification token*: Wird am Anfang jedes Satzes hinzugefügt. BERT nutzt es für Satzrepräsentationen (z. B. in Klassifikationsaufgaben). |
| `[SEP]` | *Separator token*: Trennt Sätze voneinander. Wird am Ende jedes Inputs hinzugefügt.                                                         |
| `##`    | Bedeutet: **Dieses Subtoken hängt an das vorherige an**, d. h. es steht **nicht am Wortanfang**.                                            |


Das ist die Tokenisierung eines Satzes durch BERT’s WordPiece-Tokenizer (bert-base-uncased).
BERT arbeitet nicht mit ganzen Wörtern, sondern mit Subtokens — kleineren Einheiten, die häufiger vorkommen.


Der WordPiece teilt Wörter nach Häufigkeit in Subteile, die im Trainingskorpus vorkommen.

Da „ongchoi“ selten oder unbekannt ist, wird es auf kleinere Fragmente zerlegt, die BERT kennt:

* on → häufiges Präfix (z. B. „only“, „onset“)

* \##gc → wahrscheinlich ein ungewöhnlicher Split (Kombination aus seltenem Muster)

* \##ho, \##i → weitere Substücke

Zusammen: on + gc + ho + i = „ongchoi“

Diese Splits sind nicht semantisch, sondern statistisch motiviert — sie basieren auf der Häufigkeit im Trainingsvokabular.

In [5]:
s2_tokenized = tokenizer(s2, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(s2_tokenized['input_ids'][0])
print(tokens)

['[CLS]', 'the', 'farmer', 'harvested', 'the', 'on', '##gc', '##ho', '##i', 'before', 'the', 'rainy', 'season', '.', '[SEP]']


In [6]:
def get_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    # Subtokens für „ongchoi“ finden - hard coded, geht auch eleganter
    idx = [i for i, t in enumerate(tokens) if t in ['on', '##gc' '###ho', '###i']]
    # Mittelwert über Subtokens = Wortembedding
    return outputs.last_hidden_state[0, idx, :].mean(dim=0)

In [7]:
s1 = "Ongchoi is delicious sautéed with garlic. The farmer harvested the ongchoi before the rainy season."
s2 = "The farmer harvested the ongchoi before the rainy season."

emb1, emb2 = get_embedding(s1), get_embedding(s2)
sim = cosine_similarity(emb1, emb2, dim=0)
print(f"Kosinus-Ähnlichkeit: {sim.item():.3f}")

Kosinus-Ähnlichkeit: 0.899


In [8]:
def get_embedding_for_target_word(sentence, target_word="ongchoi"):
    # Tokenisiere Satz
    inputs = tokenizer(sentence, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)

    # Tokens sichtbar machen
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])

    # Das Zielwort selbst auch tokenisieren (wie BERT es intern tun würde)
    target_tokens = tokenizer.tokenize(target_word)

    # Nun suchen wir in der Sequenz, wo genau diese Tokenfolge vorkommt:
    # len(tokens) = Länge des gesamten tokenisierten Satzes
    # len(target_tokens) = Länge des tokenisierten Zielwortes
    # Substrahiere und addiere 1 um sicherzustellen, dass wir nur die Startpositionen überprüfen, wo das Zielwort noch komplett hineinpasst
    for i in range(len(tokens) - len(target_tokens) + 1):
        # Aktueller Index + Länge des tokenisierten Zielworts
        if tokens[i:i+len(target_tokens)] == target_tokens:
          # Indices speichern
            idx = list(range(i, i+len(target_tokens)))
            break # Verlassen der Schleife, da wir gefunden haben, wonach wir suchen :)
    else:
        # Falls nicht gefunden: Rückmeldung und None
        print(f"Zielwort '{target_word}' nicht in Tokens gefunden.\nTokens: {tokens}")
        return None

    # Mittelwert über alle Subtokens → Wortembedding
    embedding = outputs.last_hidden_state[0, idx, :].mean(dim=0)
    return embedding

In [9]:
s1 = "She ate a fresh apple from the tree."
s2 = "Apple released a new iPhone this month."

target_word = 'Apple'
emb1, emb2 = get_embedding_for_target_word(s1, target_word=target_word), get_embedding_for_target_word(s2, target_word=target_word)
sim = cosine_similarity(emb1, emb2, dim=0)
print(f"Kosinus-Ähnlichkeit: {sim.item():.3f}")

Kosinus-Ähnlichkeit: 0.178


In [10]:
s1 = "The python slithered through the grass."
s2 = "She wrote a script in Python to analyze the data."

target_word = 'python'
emb1, emb2 = get_embedding_for_target_word(s1, target_word=target_word), get_embedding_for_target_word(s2, target_word=target_word)
sim = cosine_similarity(emb1, emb2, dim=0)
print(f"Kosinus-Ähnlichkeit: {sim.item():.3f}")

Kosinus-Ähnlichkeit: 0.455


In [11]:
s1 = "He ate a salty potato chip."
s2 = "The computer uses a new type of micro chip."


target_word = 'chip'
emb1, emb2 = get_embedding_for_target_word(s1, target_word=target_word), get_embedding_for_target_word(s2, target_word=target_word)
sim = cosine_similarity(emb1, emb2, dim=0)
print(f"Kosinus-Ähnlichkeit: {sim.item():.3f}")


Kosinus-Ähnlichkeit: 0.439
